In [134]:
import os
from datasets import Dataset
import soundfile as sf
import transformers
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Tokenizer, Wav2Vec2Processor
import torchaudio
import numpy as np
import torch
import traceback
from transformers import AutoProcessor
import multiprocessing

In [113]:
# # Define the feature extractor and tokenizer
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
# tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

# # Define the processor
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")

ValueError: Unrecognized processor in facebook/hubert-large-ls960-ft. Should have a `processor_type` key in its preprocessor_config.json, or one of the following `model_type` keys in its config.json: clip, flava, layoutlmv2, layoutlmv3, layoutxlm, sew, sew-d, speech_to_text, speech_to_text_2, trocr, unispeech, unispeech-sat, vilt, vision-text-dual-encoder, wav2vec2, wav2vec2-conformer, wav2vec2_with_lm, wavlm

In [135]:
def load_data(root_dir):
    data = {"path": [], "transcript": [], "split": []}

    # For each subdirectory
    for split in ["Train", "Dev", "Test"]:
        split_dir = os.path.join(root_dir, split)
        for category in os.listdir(split_dir):
            category_dir = os.path.join(split_dir, category)

            # For each file
            for file in os.listdir(category_dir):
                if file.endswith(".flac"):
                    path = os.path.join(category_dir, file)
                    transcript_path = path.replace(".flac", ".txt")

                    with open(transcript_path, "r") as f:
                        transcript = f.read().strip()

                    data["path"].append(path)
                    data["transcript"].append(transcript)
                    data["split"].append(split)

    return Dataset.from_dict(data)


def map_to_array(batch):
    speech, _ = sf.read(batch["path"])
    batch["speech"] = speech
    return batch




In [136]:
# Load data
dataset = load_data("/raid/kaisar_dauletbek/datasets/ISSAI_KSC2")

# Set the number of processes to use for parallel processing
num_processes = multiprocessing.cpu_count()

# Parallelize the mapping operation across CPUs
dataset = dataset.map(map_to_array, num_proc=num_processes)

Exception ignored in: <function _releaseLock at 0x7faf7a35beb0>
Traceback (most recent call last):
  File "/home/kaisar_dauletbek/anaconda3/envs/nlp/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


In [115]:
from datasets import load_dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 22.62it/s]
                                                             

Dataset librispeech_asr_dummy downloaded and prepared to /home/kaisar_dauletbek/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b. Subsequent calls will reuse this data.


In [119]:
ds = ds.map(map_to_array)

In [120]:
ds

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'speech'],
    num_rows: 73
})